In [1]:
import sys; print('Python %s on %s' % (sys.version, sys.platform))

sys.path.extend(['./cnn_finetune','./lib'])

from load_cifar import load_cifar10_data, load_cifar100_data

import numpy as np; print('numpy:', np.__version__)
import sklearn; print('sklearn:', sklearn.__version__)
import tensorflow as tf; print('tensorflow', tf.__version__)
import keras; print('keras', keras.__version__)

import os
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session

# set gpu visible environment variable
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

Python 3.5.2 (default, Nov 12 2018, 13:43:14) 
[GCC 5.4.0 20160609] on linux


Using TensorFlow backend.


numpy: 1.16.2
sklearn: 0.20.3
tensorflow 1.13.1
keras 2.2.4


In [2]:
config = tf.ConfigProto()

# cpu
# config = tf.ConfigProto(intra_op_parallelism_threads=num_cores, inter_op_parallelism_threads=num_cores, allow_soft_placement=True, device_count={'CPU': 4})
    
# gpu allow_growth
config.gpu_options.allow_growth = True

# set memory limit of gpu
# config.gpu_options.per_process_gpu_memory_fraction = 0.5

set_session(tf.Session(config=config))

In [3]:
from keras.optimizers import SGD, Adam
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping, ReduceLROnPlateau, TensorBoard, CSVLogger

from lib.KerasImageGenerator import *

In [4]:
# Example to fine-tune on 3000 samples from Cifar10
model_name = 'm_desenet_161'
dataset_name = 'cifar10'
img_rows, img_cols = 224, 224 # Resolution of inputs
channel = 3
num_classes = 2019
batch_size = 16
epochs = 300
load_weights = False
fine_tuning = False

In [5]:
!ls -l models

total 249124
-rw-r--r-- 1 root root 127695716 May 14 04:40 denesnet_161_20190508_2.9911_0.3629.hd5
drwxr-xr-x 3 root root      4096 May 14 07:38 densenet_161
drwxr-xr-x 3 root root      4096 May 14 08:46 densenet_161_seq
-rw-r--r-- 1 root root 127393236 May 14 04:39 imaterialist-model-saveweights.hd5
drwxr-xr-x 3 root root      4096 May 14 04:29 resnet_50


In [6]:
from cnn_finetune.densenet1612019 import densenet161_model

In [7]:
# weights_path="./models/denesnet_161_20190508_2.9911_0.3629.hd5"

weights_path="./models/imaterialist-model-saveweights.hd5"

In [8]:
# Load our model
val_model = densenet161_model(img_rows, img_cols, channel, 
                          num_classes=num_classes, fine_tuning=fine_tuning, load_weights=load_weights,
                          )

Instructions for updating:
Colocations handled automatically by placer.
dense161_model: classes: 2019 fine-tuning: False
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
data (InputLayer)               (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_zeropadding (ZeroPadding2 (None, 230, 230, 3)  0           data[0][0]                       
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 96) 14112       conv1_zeropadding[0][0]          
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 96) 384         conv1[0][0]           

In [9]:
val_model.load_weights(weights_path, by_name=True)

In [10]:
val_Image_Keras_Generator = KerasImageGenerator(ds='val',corp = True,batch_size = 16, shuffle=False)
val_image_generator_Iterator = val_Image_Keras_Generator.keras_get_images_iter()
val_image_generator_Iterator.reset()
print(len(val_image_generator_Iterator))

Found 10095 images belonging to 2019 classes.
631


In [11]:
help(val_image_generator_Iterator)

Help on DirectoryIterator in module keras_preprocessing.image.directory_iterator object:

class DirectoryIterator(keras_preprocessing.image.iterator.BatchFromFilesMixin, keras_preprocessing.image.iterator.Iterator)
 |  Iterator capable of reading images from a directory on disk.
 |  
 |  # Arguments
 |      directory: string, path to the directory to read images from.
 |          Each subdirectory in this directory will be
 |          considered to contain images from one class,
 |          or alternatively you could specify class subdirectories
 |          via the `classes` argument.
 |      image_data_generator: Instance of `ImageDataGenerator`
 |          to use for random transformations and normalization.
 |      target_size: tuple of integers, dimensions to resize input images to.
 |      color_mode: One of `"rgb"`, `"rgba"`, `"grayscale"`.
 |          Color mode to read images.
 |      classes: Optional list of strings, names of subdirectories
 |          containing images from 

In [12]:
from lib.DataGenerator import DataGenerator
val_data_generator = DataGenerator(file_type='val', corp=True, batch_size=batch_size, shuffle=False)
# train_data_generator = DataGenerator(file_type='train', corp=False, batch_size=batch_size)

Found 10095 images belonging to 2019 classes.


In [13]:
val_prediction = val_model.predict_generator(val_data_generator, 
                                     steps = len(val_data_generator),
                                     workers=50, use_multiprocessing=True,
                                     verbose=1)

631/631 [==============================] - 53s 83ms/step


In [14]:
val_filenames = val_data_generator.image_iterator.filenames

In [15]:
# val_filenames1 = val_image_generator_Iterator.filenames

In [16]:
# val_prediction = val_model.predict_generator(val_image_generator_Iterator, 
#                                      steps = len(val_image_generator_Iterator),
#                                      workers=1, use_multiprocessing=False,
#                                      verbose=1)

In [17]:
# val_filenames = val_image_generator_Iterator.filenames

In [18]:
print(val_filenames1 == val_filenames)

NameError: name 'val_filenames1' is not defined

In [19]:
import copy
val_predictions = copy.deepcopy(val_prediction)

In [20]:
print(val_predictions.shape)

(10095, 2019)


In [21]:
# load val id class file
import json
with open("./preprocess/val_id_class.json") as f:
    val_id_class_json = json.load(f)

In [22]:
cnt = 0
for idx in range(len(val_filenames)):
#     print((val_id_class_json.get(val_filenames[idx].split('/')[-1])))
#     print('>>>', val_predictions[idx].shape)
    print(int(np.argmax(val_predictions[idx])))
    if int(val_id_class_json.get(val_filenames[idx].split('/')[-1])) == int(np.argmax(val_predictions[idx])):
        cnt = cnt+1

0
0
0
0
0
1
1
1
1
1
2
2
2
2
2
3
3
3
3
3
4
4
4
4
4
5
5
5
5
5
6
6
6
6
6
7
7
7
7
7
8
8
8
8
8
9
9
9
9
9
10
10
10
10
10
11
11
11
11
11
12
12
12
12
12
13
13
13
13
13
14
14
14
14
14
15
15
15
15
15
16
16
16
16
16
17
17
17
17
17
18
18
18
18
18
19
19
19
19
19
20
20
20
20
20
21
21
21
21
21
22
22
22
22
22
23
23
23
23
23
24
24
24
24
24
25
25
25
25
25
26
26
26
26
26
27
27
27
27
27
28
28
28
28
28
29
29
29
29
29
30
30
30
30
30
31
31
31
31
31
32
32
32
32
32
33
33
33
33
33
34
34
34
34
34
35
35
35
35
35
36
36
36
36
36
37
37
37
37
37
38
38
38
38
38
39
39
39
39
39
40
40
40
40
40
41
41
41
41
41
42
42
42
42
42
43
43
43
43
43
44
44
44
44
44
45
45
45
45
45
46
46
46
46
46
47
47
47
47
47
48
48
48
48
48
49
49
49
49
49
50
50
50
50
50
51
51
51
51
51
52
52
52
52
52
53
53
53
53
53
54
54
54
54
54
55
55
55
55
55
56
56
56
56
56
57
57
57
57
57
58
58
58
58
58
59
59
59
59
59
60
60
60
60
60
61
61
61
61
61
62
62
62
62
62
63
63
63
63
63
64
64
64
64
64
65
65
65
65
65
66
66
66
66
66
67
67
67
67
67
68
68
68
68
68
69
69
69
69
69


663
664
664
664
664
664
665
665
665
665
665
666
666
666
666
666
667
667
667
667
667
668
668
668
668
668
669
669
669
669
669
670
670
670
670
670
671
671
671
671
671
672
672
672
672
672
673
673
673
673
673
674
674
674
674
674
675
675
675
675
675
676
676
676
676
676
677
677
677
677
677
678
678
678
678
678
679
679
679
679
679
680
680
680
680
680
681
681
681
681
681
682
682
682
682
682
683
683
683
683
683
684
684
684
684
684
685
685
685
685
685
686
686
686
686
686
687
687
687
687
687
688
688
688
688
688
689
689
689
689
689
690
690
690
690
690
691
691
691
691
691
692
692
692
692
692
693
693
693
693
693
694
694
694
694
694
695
695
695
695
695
696
696
696
696
696
697
697
697
697
697
698
698
698
698
698
699
699
699
699
699
700
700
700
700
700
701
701
701
701
701
702
702
702
702
702
703
703
703
703
703
704
704
704
704
704
705
705
705
705
705
706
706
706
706
706
707
707
707
707
707
708
708
708
708
708
709
709
709
709
709
710
710
710
710
710
711
711
711
711
711
712
712
712
712
712
713
713
713
713


1288
1288
1288
1289
1289
1289
1289
1289
1290
1290
1290
1290
1290
1291
1291
1291
1291
1291
1292
1292
1292
1292
1292
1293
1293
1293
1293
1293
1294
1294
1294
1294
1294
1295
1295
1295
1295
1295
1296
1296
1296
1963
1296
1297
1297
1297
1297
1297
1298
1298
1298
1298
1298
1299
1299
1299
1299
1299
1300
1300
1300
1300
1300
1301
1301
1301
1301
1301
1302
1302
1302
1302
1302
1303
1303
1303
1303
1303
1304
1304
1304
1304
1304
1305
1305
1305
1305
1305
1306
1306
1306
1306
1306
1307
1307
1307
1307
1307
1308
1308
1308
1308
1308
1309
1309
1309
1309
1309
1310
1310
1310
1310
1310
1311
1311
1311
1311
1311
1312
1312
1312
1312
1312
1313
1313
1313
1313
1313
1314
1314
1314
1314
1314
1315
1315
1315
1315
1315
1316
1316
1316
1316
1316
1317
1317
1317
1317
1317
1318
1318
1318
1318
1318
1319
1319
1319
1319
1319
1320
1320
1320
1320
1320
1321
1321
1321
1321
1321
1322
1322
1322
1322
1322
1323
1323
1323
1323
1323
1324
1324
1324
1324
1324
1325
1325
1325
1325
1325
1326
1326
1326
1326
1326
1327
1327
1327
1327
1327
1328
1328


1669
1669
1670
1670
1670
1670
1670
1671
1671
1671
1671
1671
1672
1672
1672
1672
1672
1673
1673
1673
1673
1673
1674
1674
1674
1674
1674
1675
1675
1675
1675
1675
1676
1676
1676
1676
1676
1677
1677
1677
1677
1677
1678
1678
1678
1678
1678
1679
1679
1679
1679
1679
1680
1680
1680
1680
1680
1681
1681
1681
1681
1681
1682
1682
1682
1682
1682
1683
1683
1683
1683
1683
1684
1684
1684
1684
1684
1685
1685
1685
1685
1685
1686
1686
1686
1686
1686
1687
1687
1687
1687
1687
1688
1688
1688
1688
1688
1689
1689
1689
1689
1689
1690
1690
1690
1690
1690
1691
1691
1691
1691
1691
1692
1692
1692
1692
1692
1693
1693
1693
1693
1693
1694
1694
1694
1694
1694
1695
1695
1695
1695
1695
1696
1696
1696
1696
1696
1697
1697
1697
1697
1697
1698
1698
1698
1698
1698
1699
1699
1699
1699
1699
1700
1700
1700
1700
1700
1701
1701
1701
1701
1701
1702
1702
1702
1702
1702
1703
1703
1703
1703
1703
1704
1704
1704
1704
1704
1705
1705
1705
1705
1705
1706
1706
1706
1706
1706
1707
1707
1707
1707
1707
1708
1708
1708
1708
1708
1709
1709
1709


In [23]:
print(cnt*1.00/len(val_filenames))

0.9998018821198613
